<a href="https://colab.research.google.com/github/monerahalmobarak/Faster-RCNN/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
from torchvision import transforms as T
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random
from PIL import Image, ImageDraw
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
import pandas as pd

# Assuming 'dataframe' is your DataFrame containing the image data
df = pd.read_csv("/content/drive/MyDrive/train.csv")

# Generate the unique image identifiers
unique_imgs = df['ImageID'].unique()

In [ ]:
df

,ImageID,XMin,XMax,YMin,YMax,ClassName,bbox
0,Accident_15_50.jpg,177,202,152,184,Accident,"[177, 202, 152, 184]"
1,Accident_12_50.jpg,180,203,150,177,Accident,"[180, 203, 150, 177]"
2,Accident_1_41.jpg,743,931,314,391,Accident,"[743, 931, 314, 391]"
3,Accident_22_50.jpg,140,171,183,200,Accident,"[140, 171, 183, 200]"
4,Accident_4_41.jpg,785,963,311,381,Accident,"[785, 963, 311, 381]"
...,...,...,...,...,...,...,...
254,Non_Accident_12_45.jpg,705,1041,329,467,Non Accident,"[705, 1041, 329, 467]"
255,Non_Accident_3_42.jpg,246,527,383,509,Non Accident,"[246, 527, 383, 509]"
256,Non_Accident_13_39.jpg,798,1005,362,437,Non Accident,"[798, 1005, 362, 437]"
257,Non_Accident_22_42.jpg,404,746,272,385,Non Accident,"[404, 746, 272, 385]"


In [ ]:
class CustDat(torch.utils.data.Dataset):
    def __init__(self, dataframe, unique_imgs, indices, transforms=None):
        self.dataframe = dataframe
        self.unique_imgs = unique_imgs
        self.indices = indices
        self.transforms = transforms  # Optional transforms
        self.class_to_idx = {'Non Accident': 0, 'Accident': 1}

    def __getitem__(self, idx):
        image_name = self.unique_imgs[self.indices[idx]]
        df_filtered = self.dataframe[self.dataframe['ImageID'] == image_name]

        # Ensure at least one result is found
        if df_filtered.empty:
            raise ValueError(f"No matching entry found for ImageID: {image_name}")

        boxes = df_filtered[['XMin', 'YMin', 'XMax', 'YMax']].values.astype('float32')
        labels = df_filtered['ClassName'].apply(lambda x: self.class_to_idx[x]).tolist()

        image_path = os.path.join("/content/drive/MyDrive/last", image_name)
        img = Image.open(image_path).convert("RGB")

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.indices)

In [ ]:
!pip install pycocotools

In [ ]:
def custom_collate(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return images, targets

In [ ]:
# Define your transforms
transform = T.Compose([T.ToTensor()])

# Use all images in the dataset
indices = range(len(unique_imgs))

# Create the dataset with all images included
dataset = CustDat(df, unique_imgs, indices, transforms=transform)

# Now you can create the DataLoader with the dataset
train_d1 = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=custom_collate, pin_memory=True if torch.cuda.is_available() else False)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 3
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
print(len(train_d1))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
num_epochs = 100

In [ ]:
import os

In [ ]:
print("Starting training...")

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} started...")
    model.train()  # Set the model to training mode
    epoch_loss = 0  # To accumulate loss over the epoch

    for images, targets in train_d1:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)

        # Total loss
        losses = sum(loss for loss in loss_dict.values())

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Backward pass + optimize
        losses.backward()
        optimizer.step()

        # Update epoch loss
        epoch_loss += losses.item()

    # Print the loss for the current epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}')

Starting training...
Epoch 1 started...
Epoch 1/100, Loss: 0.29144128039479256
Epoch 2 started...
Epoch 2/100, Loss: 0.3010373776778579
Epoch 3 started...
Epoch 3/100, Loss: 0.3089988911524415
Epoch 4 started...
Epoch 4/100, Loss: 0.2950144475325942
Epoch 5 started...
Epoch 5/100, Loss: 0.2951991055160761
Epoch 6 started...
Epoch 6/100, Loss: 0.2885185182094574
Epoch 7 started...
Epoch 7/100, Loss: 0.30756924860179424
Epoch 8 started...
Epoch 8/100, Loss: 0.3019648250192404
Epoch 9 started...
Epoch 9/100, Loss: 0.2932438552379608
Epoch 10 started...
Epoch 10/100, Loss: 0.29572868533432484
Epoch 11 started...
Epoch 11/100, Loss: 0.29459198750555515
Epoch 12 started...
Epoch 12/100, Loss: 0.30782063119113445
Epoch 13 started...
Epoch 13/100, Loss: 0.29740296863019466
Epoch 14 started...
Epoch 14/100, Loss: 0.2924417480826378
Epoch 15 started...
Epoch 15/100, Loss: 0.295561708509922
Epoch 16 started...
Epoch 16/100, Loss: 0.2939570154994726
Epoch 17 started...
Epoch 17/100, Loss: 0.290636

In [ ]:
# Load the test data
test_df = pd.read_csv("/content/test.csv")

# Assuming you have the same structure for test data and you've defined the necessary transformations
test_unique_imgs = test_df['ImageID'].unique()

In [ ]:
# Create the dataset for testing
test_indices = range(len(test_unique_imgs))
test_dataset = CustDat(test_df, test_unique_imgs, test_indices, transforms=transform)

In [ ]:
# Create the DataLoader for testing
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate, pin_memory=True if torch.cuda.is_available() else False)

In [ ]:
def calculate_iou(box1, box2):
    """Calculate Intersection over Union (IoU) of two bounding boxes."""
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1 = max(x1, x1g)
    yi1 = max(y1, y1g)
    xi2 = min(x2, x2g)
    yi2 = min(y2, y2g)

    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area

    return iou

def evaluate(model, dataloader, iou_threshold=0.8):
    model.eval()
    total_detections = 0
    total_correct = 0

    with torch.no_grad():
        for images, targets in dataloader:
            images = list(image.to(device) for image in images)
            outputs = model(images)

            for i, output in enumerate(outputs):
                pred_boxes = output['boxes']
                pred_scores = output['scores']
                pred_labels = output['labels']

                true_boxes = targets[i]['boxes'].to(device)
                true_labels = targets[i]['labels'].to(device)

                # Filter predictions with a confidence score threshold
                high_conf_indices = pred_scores > 0.8
                pred_boxes = pred_boxes[high_conf_indices]
                pred_labels = pred_labels[high_conf_indices]

                for pred_box, pred_label in zip(pred_boxes, pred_labels):
                    # Here we assume one ground truth per label for simplicity
                    # In practice, you might need to match labels too
                    for true_box, true_label in zip(true_boxes, true_labels):
                        if pred_label == true_label:
                            iou = calculate_iou(pred_box.cpu(), true_box.cpu())
                            if iou >= iou_threshold:
                                total_correct += 1
                            break

                total_detections += len(pred_boxes)

    accuracy = total_correct / total_detections if total_detections > 0 else 0
    return accuracy

# Evaluate the model
test_accuracy = evaluate(model, test_dataloader)
print(f"Test Accuracy: {test_accuracy}")